In [19]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import dynet as dy
import pickle

In [20]:
labels = pickle.load(open("../corpus/labels_40000.pkl"))


Training part

In [21]:
sentence_model_filename = "../models/sentence_structure_40k.npz"
def load_model():
    with np.load(sentence_model_filename) as data:
        sent_indices = data["sent_indices"]
        sent_mask = data["sent_mask"]
    return sent_indices, sent_mask

In [51]:
sent_indices, sent_mask = load_model()
sent_count = sent_indices.shape[0]
max_length = sent_indices.shape[1]
print(sent_count, max_length)

(808580, 245)


In [35]:
empty_index1 = np.argwhere(np.sum(sent_mask[sent_count//2:sent_count,:], axis = 1, dtype = int)== 0)
empty_index2 = np.argwhere(np.sum(sent_mask[:sent_count//2,:], axis = 1, dtype = int) == 0)

In [50]:
l = np.union1d(empty_index1,empty_index2)
empty_count = l.shape[0]
sent_count -= 2*empty_count
sent_count

808398

In [46]:
sent_ind = np.zeros(sent_count,max_length)
sent_masks = np.zeros(sent_count,max_length)
sent_ind[0:sent_count//2,:] = np.delete(sent_indices[:sent_count//2], l, axis = 0)
sent_ind[sent_count//2:sent_count,:] = np.delete(sent_indices[sent_count//2:sent_count], l, axis = 0)
sent_masks[0:sent_count//2 - empty_count,:] = np.delete(sent_mask[:sent_count//2], l, axis = 0)
sent_masks[sent_count//2:sent_count - empty_count,:] = np.delete(sent_mask[sent_count//2:sent_count], l, axis = 0)
labels = np.delete(labels, l, axis = 0)

In [49]:
print(sent_ind.shape)
print(sent_masks[-1,:])
print(labels.shape)

(808580, 245)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [15]:
#def similarity():
    
#data is already in data numpy matrix

class decomposable_attention():
    def __init__(self,sent_indices, sent_mask, labels, embedding_dim, debug=False):
        self.pc = dy.Model()
        self.debug = debug
        self.embedding_dim = embedding_dim 
        self.embedding_matrix = self.pc.add_lookup_parameters((16836,embedding_dim))
        self.data1 = sent_indices[:404290,:]#np.random.randint(16800,size=(10000,50))
        self.data2 = sent_indices[404290:20000,:]#np.random.randint(16800,size=(10000,50))
        self.len1 = np.sum(sent_mask[:404290,:], axis = 1, dtype = int)#1+np.random.randint(50,size=10000)
        self.len2 = np.sum(sent_mask[404290:20000,:], axis = 1, dtype = int)#1+np.random.randint(50,size=10000)
        self.labels = labels
        
        self.w2 = self.pc.add_parameters((embedding_dim,2*embedding_dim))
        self.b2 = self.pc.add_parameters((embedding_dim,1))
        
        self.w3 = self.pc.add_parameters((embedding_dim,2*embedding_dim))
        self.b3 = self.pc.add_parameters((embedding_dim,1))
        
        self.w = self.pc.add_parameters((1,2*embedding_dim))
        self.b = self.pc.add_parameters((1,1))
        
        
    def forward(self,x1,x2,label,k,tl):
        
        debug = self.debug
        len1 = self.len1
        len2 = self.len2
        embedding_dim = self.embedding_dim
        
        w = dy.parameter(self.w)
        b = dy.parameter(self.b)
        w2 = dy.parameter(self.w2)
        b2 = dy.parameter(self.b2)
        w3 = dy.parameter(self.w3)
        b3 = dy.parameter(self.b3)
        
        embeds1 = dy.reshape(dy.lookup_batch(self.embedding_matrix,x1),(len1[k],embedding_dim))
        embeds2 = dy.reshape(dy.lookup_batch(self.embedding_matrix,x2),(len2[k],embedding_dim))

        if debug:
            print('embedding 1:', (len1[k],embedding_dim), embeds1.dim())
            print('embedding 2:', (embedding_dim,len2[k]), embeds2.dim())

        similarity = embeds1*dy.transpose(embeds2)
        if debug:
            print('similarity dimension:', (len1[k],len2[k]), similarity.dim())

        n_a = dy.softmax(similarity,d=0)
        n_b = dy.softmax(similarity,d=1)
        
        alpha = dy.transpose(n_a)*embeds1
        beta = n_b*embeds2
        
        if debug:
            print('alpha:',(len2[k],embedding_dim), alpha.dim())
            print('beta:',(len1[k],embedding_dim), beta.dim())
        
        #print((w2*dy.transpose(dy.concatenate_cols([embeds1,beta]))).npvalue().shape)
        #print(b2.npvalue().shape)
        
        v1i = w2*dy.transpose(dy.concatenate_cols([embeds1,beta])) + b2
        v2j = w3*dy.transpose(dy.concatenate_cols([embeds2,alpha])) + b3
        if debug:
            print('v1:', (embedding_dim,len1[k]), v1i.dim())
            print('v2:', (embedding_dim,len2[k]), v2j.dim())
        
        v1 = dy.mean_dim(v1i,[1],0)
        v1 = dy.reshape(v1,(embedding_dim,1)) 
        
        v2 = dy.mean_dim(v2j,[1],0)
        v2 = dy.reshape(v2,(embedding_dim,1)) 
        
        score = w*dy.concatenate([v1,v2],d=0) + b
        
        return score
        
    def train(self):
        len1 = self.len1
        len2 = self.len2
        embedding_dim = self.embedding_dim
        lr = 0.001
        trainer = dy.AdamTrainer(self.pc, alpha = lr)
        
        iter = 0
        for epochs in range(100):
            tl = 0
            for k in range(len(self.labels)):
                iter += 1
                dy.renew_cg()
                x1 = self.data1[k,0:len1[k]]
                x2 = self.data2[k,0:len2[k]]
                label = self.labels[k]

                score = self.forward(x1,x2,label,k,tl)
                norm_score = dy.logistic(score)
                
                loss = dy.binary_log_loss(norm_score,dy.inputTensor([[label]]))
                loss.backward()
                trainer.update()
                tl += loss.scalar_value()
                #if iter % dev_iter == 0:
                    #predict_fn
            if epochs in [5,15,30,50] :
                lr /= 2
                trainer = dy.AdamTrainer(self.pc, alpha = lr)
                    
            print('Train loss after ' + str(epochs+1) + ' epochs: ' + str(tl/len(self.labels)))

def main():
    
    model = decomposable_attention(sent_indices, sent_mask, labels, embedding_dim=128, debug=False)
    model.train()
    
main()           

ValueError: CwiseSum: For each dimension, the dim size needs to match or equal 1.

In [10]:
x1 = np.random.randint(5,size=10)
x2 = np.random.randint(5,size=15)

pc = dy.Model()
embedding_matrix = pc.add_lookup_parameters((168000,222))
a = dy.lookup_batch(embedding_matrix,x1)
b = dy.lookup_batch(embedding_matrix,x2)

c = a*b
print(c.dim())

ValueError: Mismatched input dimensions in MatrixMultiply: [{222X10} {222X15}]

In [28]:

pc = dy.Model()
s = pc.add_parameters((4))
s1 = pc.add_parameters((4))

dy.renew_cg()
sa = dy.parameter(s)
sb = dy.parameter(s1)
na = dy.concatenate([sa,sb])
na.npvalue()
#n_b = dy.softmax(similarity,d=1)

array([ 0.48167747, -0.07805443, -0.22833993,  0.49472263, -0.24113466,
        0.788912  ,  0.25677112, -0.74309444])

In [42]:
def encoder(document, W_cnn, b_cnn, W, b):
        dy.renew_cg()
        W_cnn = dy.parameter(W_cnn)
        b_cnn = dy.parameter(b_cnn)
        W = dy.parameter(W)
        b = dy.parameter(b)


        cnn_in = dy.concatenate([dy.lookup(lookup,x) for x in document], d=1)
        cnn_out = dy.conv2d_bias(cnn_in, W_cnn, b_cnn, stride=(1, 2), is_valid=False)
        cnn_out = dy.conv2d_bias(cnn_out, W, b, stride = (1,2), is_valid=False)
        pool_out = dy.mean_dim(cnn_out, d=[1], b=0)
        print(pool_out.npvalue().shape)
        pool_out = dy.reshape(pool_out, (32,))
        pool_out = dy.rectify(pool_out)

        return pool_out

In [43]:
len1 = np.sum(sent_mask[:10000,:], axis = 1, dtype = int)
len2 = np.sum(sent_mask[10000:20000,:], axis = 1, dtype = int)

pc = dy.Model()
W_cnn = pc.add_parameters((1, 2, 128, 64)) # cnn weights
b_cnn = pc.add_parameters((64))
W = pc.add_parameters((1, 2, 64, 32)) # cnn weights
b = pc.add_parameters((32))
lookup = pc.add_lookup_parameters((20000, 1, 1, 128))
for k in range(100):
    document = sent_indices[k][0:len1[k]]
    z = encoder(document, W_cnn, b_cnn, W, b)
#     print(z.npvalue().shape)
                

(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
(1, 32)
